In [ ]:
###### setup #######

from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot
from jetbot import bgr8_to_jpeg
import ipywidgets.widgets as widgets
import cv2
import numpy as np
import time

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300, fps=5)
robot = Robot()

In [ ]:
###### definitions #######

target = 1 # arbitrary final target (human)
obstacle = 37 # arbitrary obstacle (sports ball)
obstacle2 = 84 # book
isEndReached = False
first = True

objectDictionary = {1:('human', 200), 3:('ball', 37)}

In [ ]:
####### other utilities #########

imageFeed = widgets.Image(format='jpeg', width=300, height=300)
speedSlider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, description='speed')

width = int(imageFeed.width)
height = int(imageFeed.height)

# display(widgets.VBox([widgets.HBox([imageFeed]), speedSlider]))
    # moved down during testing, bring back up or just delete later on

In [ ]:
###### non essemtial helpers, not finalized ########
    
def leftTurn(incremental):
    robot.left(speedSlider.value) # arbitrary small value
    if incremental:
        time.sleep(0.125) # can change, might want to go higher
    else:
        time.sleep(0.075)
    robot.stop()
    
def rightTurn(incremental):
    robot.right(speedSlider.value) # arbitrary small value
    if incremental:
        time.sleep(0.125) # can change, might want to go higher
    else:
        time.sleep(0.075)
    robot.stop()
    
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

In [ ]:
###### essential helpers + main ########

display(widgets.VBox([widgets.HBox([imageFeed]), speedSlider]))
    # just here to see easier when test; remove later on

def center(detection, image):
    
    # non essential, delete later?
    imageFeed.value = bgr8_to_jpeg(image)
    
    center = detection_center(detection)
    xOffset = center[0]
    
    # to make more accurate, should scale offsets with distance from target object
    if xOffset > 0.125:
        rightTurn(False)
        lockOn(camera.value, True)
    elif xOffset < -0.125:
        leftTurn(False)
        lockOn(camera.value, True) 
    else:
        print('centered')
        
def avoid(bbox):
    # Move back a bit
    if int(height * bbox[3]) > height - 50:
        robot.stop()
        robot.backward(speedSlider.value)
                
    # If the object is now out of collision zone, avoid obstacle
    if int(height * bbox[3]) >= height - 40:
        print('backing')
    
        robot.stop()
    
        robot.left(speedSlider.value)
        time.sleep(0.4)
        robot.stop()
    
        robot.left_motor.value = 0.3
        robot.right_motor.value = 0.2
        time.sleep(1.5)
        robot.stop()
        
def lookingToAvoid(image):
    detections = model(image)
    for detection in detections[0]:
        bbox = detection['bbox']
        label = detection['label']
        if label == obstacle or label == obstacle2:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            avoid(bbox)
        
def notInVision(isCentering):
    inVision = False
    
    while(not inVision): 
        rightTurn(True)
        time.sleep(0.25)
        
        image = camera.value
        detections = model(image)

        for detection in detections[0]:        
            bbox = detection['bbox']
            label = detection['label']
            if label == target:
                cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
                # print('detected after turning')
                center(detection, image)
                inVision = True
                break
                
        imageFeed.value = bgr8_to_jpeg(camera.value)

def lockOn(image, isCentering):
    isFound = False
    detections = model(image)
    
    for detection in detections[0]:        
        label = detection['label']
        if label == target:
            # print('in view initially; detected')
            isFound = True
            center(detection, image)
            break
    
    if not isFound:
        notInVision(isCentering)
        
    if not isCentering:
                
        imageFeed.value = bgr8_to_jpeg(camera.value)
        
        robot.forward(0.3)
        
        global first 
        first = False

def run(change):
    image = change['new']
    
    global first
    if not first:
        lookingToAvoid(image)
    
    imageFeed.value = bgr8_to_jpeg(image)
            
    lockOn(camera.value, False)
    
    lookingToAvoid(camera.value)

In [ ]:
###### running ########

# while not isEndReached:
for i in range (0,20):    
    run({'new': camera.value})
    global first
    print(first)
    
print('end of the loop (maybe the target was reached?)')
robot.stop()

# was having issues with the following
# the camera would use too much of the memory and would not update, took the easy way out 
#            and looped "run({'new': camera.value})" 
# https://github.com/NVIDIA-AI-IOT/jetbot/issues/120
# https://github.com/NVIDIA-AI-IOT/jetbot/issues/63

# camera.unobserve_all()
# camera.observe(run, names='value')